In [1]:
# 0.0 import packages

import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# 1.0 load in datasets

def load_datasets(data_id, print_out = False, train_device="cpu", val_device="cpu", test_device="cpu"):
    # load files
    train_feats = np.load("data/dataset" + data_id + "/train_feats" + data_id + ".npy", allow_pickle=True)
    train_labels = np.load("data/dataset" + data_id + "/train_labels" + data_id + ".npy", allow_pickle=True)
    test_feats = np.load("data/dataset" + data_id + "/test_feats" + data_id + ".npy", allow_pickle=True)
    test_labels = np.load("data/dataset" + data_id + "/test_labels" + data_id + ".npy", allow_pickle=True)
    val_feats = np.load("data/dataset" + data_id + "/val_feats" + data_id + ".npy", allow_pickle=True)
    val_labels = np.load("data/dataset" + data_id + "/val_labels" + data_id + ".npy", allow_pickle=True)
    # reshape numpy arrays
    train_feats = train_feats.reshape(train_feats.shape[0]*train_feats.shape[1], train_feats.shape[2])
    train_labels = train_labels.reshape(train_labels.shape[0]*train_labels.shape[1], train_labels.shape[2])
    test_feats = test_feats.reshape(test_feats.shape[0]*test_feats.shape[1], test_feats.shape[2])
    test_labels = test_labels.reshape(test_labels.shape[0]*test_labels.shape[1], test_labels.shape[2])
    val_feats = val_feats.reshape(val_feats.shape[0]*val_feats.shape[1], val_feats.shape[2])
    val_labels = val_labels.reshape(val_labels.shape[0]*val_labels.shape[1], val_labels.shape[2])
    
    if print_out:
        print("train feats: ", train_feats.shape, " type: ", type(train_feats))
        print("train labels: ", train_labels.shape, " type: ", type(train_labels))

        print("test feats: ", test_feats.shape, " type: ", type(test_feats))
        print("test labels: ", test_labels.shape, " type: ", type(test_labels))

        print("val feats: ", val_feats.shape, " type: ", type(val_feats))
        print("val labels: ", val_labels.shape, " type: ", type(val_labels))

    # create tensors
    train_feats_tensor = torch.tensor(train_feats, requires_grad=True, dtype=torch.float).to(train_device)
    train_labels_tensor = torch.tensor(train_labels, dtype=torch.float).to(train_device)

    test_feats_tensor = torch.tensor(test_feats, requires_grad=True, dtype=torch.float).to(test_device)
    test_labels_tensor = torch.tensor(test_labels, dtype=torch.float).to(test_device)

    val_feats_tensor = torch.tensor(val_feats, requires_grad=True, dtype=torch.float).to(val_device)
    val_labels_tensor = torch.tensor(val_labels, dtype=torch.float).to(val_device)

    if print_out:
        print ("train feats tensor: ", train_feats_tensor.shape, " type: ", type(train_feats_tensor))
        print ("train labels tensor: ", train_labels_tensor.shape, " type: ", type(train_labels_tensor))

        print ("test feats tensor: ", test_feats_tensor.shape, " type: ", type(test_feats_tensor))
        print ("test labels tensor: ", test_labels_tensor.shape, " type: ", type(test_labels_tensor))

        print ("val feats tensor: ", val_feats_tensor.shape, " type: ", type(val_feats_tensor))
        print ("val labels tensor: ", val_labels_tensor.shape, " type: ", type(val_labels_tensor))
            
        if print_out:
            print ("train_feats_tensor.device:", train_feats_tensor.get_device())
            print ("train_labels_tensor.device:", train_labels_tensor.get_device())
            print ("test_feats_tensor.device:", test_feats_tensor.get_device())
            print ("test_labels_tensor.device:", test_labels_tensor.get_device())
            print ("val_feats_tensor.device:", val_feats_tensor.get_device())
            print ("val_labels_tensor.device:", val_labels_tensor.get_device())
        
    return  (train_feats_tensor,
            train_labels_tensor,
            test_feats_tensor,
            test_labels_tensor,
            val_feats_tensor,
            val_labels_tensor)

In [3]:
# 2.0 set up the data loaders

# tensor tuple shape: output of load_datasets()
#   [ train_feats_tensor, train_labels_tensor,
#     test_feats_tensor, test_labels_tensor,
#     val_feats_tensor, val_labels_tensor ]
def set_up_dataloaders(batch_size, tensor_tuple):

    train_dataset = torch.utils.data.TensorDataset(tensor_tuple[0], tensor_tuple[1])
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle = True)

    test_dataset = torch.utils.data.TensorDataset(tensor_tuple[2], tensor_tuple[3])
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle = False)

    val_dataset = torch.utils.data.TensorDataset(tensor_tuple[4], tensor_tuple[5])
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle = False)
    
    return  (train_dataset, train_loader,
            test_dataset, test_loader,
            val_dataset, val_loader)

In [ ]:
class my_model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(my_model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [ ]:
# create model
model = my_model(128, 9, 32, 2)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_step(batch):
    pass

def test_model(model):
    pass

In [ ]:
def train(dataset, epochs):
  # A. For each epoch, do the following:
  for epoch in range(epochs):
    start = time.time()
    # 1 - For each batch of the epoch, 
    for batch in dataset:
      # 1.a - run the custom "train_step" function
      # we just declared above
      train_step(batch)

    # 4 - Print out the completed epoch no. and the time spent
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  test_model()

In [4]:
# 4.0 model training

# training parameters
batch_size = 10
num_epochs = 5

# get dataloaders
train_dataset, train_loader, test_dataset, test_loader, val_dataset, val_loader  = set_up_dataloaders(
    batch_size, load_datasets(data_id="_5s_50hz", print_out=True)
)

train(train_dataset, num_epochs)

train feats:  (22250, 128)  type:  <class 'numpy.ndarray'>
train labels:  (22250, 9)  type:  <class 'numpy.ndarray'>
test feats:  (230250, 128)  type:  <class 'numpy.ndarray'>
test labels:  (230250, 9)  type:  <class 'numpy.ndarray'>
val feats:  (237000, 128)  type:  <class 'numpy.ndarray'>
val labels:  (237000, 9)  type:  <class 'numpy.ndarray'>
train feats tensor:  torch.Size([22250, 128])  type:  <class 'torch.Tensor'>
train labels tensor:  torch.Size([22250, 9])  type:  <class 'torch.Tensor'>
test feats tensor:  torch.Size([230250, 128])  type:  <class 'torch.Tensor'>
test labels tensor:  torch.Size([230250, 9])  type:  <class 'torch.Tensor'>
val feats tensor:  torch.Size([237000, 128])  type:  <class 'torch.Tensor'>
val labels tensor:  torch.Size([237000, 9])  type:  <class 'torch.Tensor'>
train_feats_tensor.device: -1
train_labels_tensor.device: -1
test_feats_tensor.device: -1
test_labels_tensor.device: -1
val_feats_tensor.device: -1
val_labels_tensor.device: -1
